In [3]:
import nilearn as nl
from nilearn import datasets

/opt/anaconda3/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [4]:
nl.datasets

<module 'nilearn.datasets' from '/opt/anaconda3/lib/python3.8/site-packages/nilearn/datasets/__init__.py'>